# Application 1:  a simple nested cube with no preconditioner

In [1]:
# PARAMETERS

## PHYSICS
k = {
    'e': 1,
    'i': 1,
    'w': 1,
}

mu = 1

## COMPUTING
element_size = 0.5

length_cavity = 1
length_wall = 2

DOMAIN_DISC_CHOICE = "RWG"
DUAL_DISC_CHOICE = "RBC"

# Imports

In [2]:
import time
from collections import namedtuple
import numpy as np
import bempp.api

from bempp.api import shapes
from bempp.api import function_space, assembly
from bempp.api.operators.boundary import maxwell
from bempp.api.operators.potential import maxwell as maxwell_potential


from bempp.core.common.global_parameters import global_parameters # Interface to define global_parameters for operators

bempp.api.set_ipython_notebook_viewer()

# Create grid

This problem domain consists of two cubes: one large and one small. The small cube is positioned at the center of the large cube.

In [3]:
def discretize(grid, *methods):
    """
    Returns a named tuple containing the discretised boundaries of the
    given grid according to the method specified.
    """
    supported_methods = ["RWG", "RBC"]
    space = {}
    assert len(methods) > 0, "You must provide disretisation methods."
    for method in methods:
        if method not in supported_methods:
            raise ValueError("Method %s not supported." % method)
        space[method] = function_space(grid, method, 0)
    return space

# Cubes are positioned according to their bottom left corner
small_cube = discretize(
    shapes.cube(length=length_cavity, h=element_size,
                origin=(-length_cavity/2, -length_cavity/2, -length_cavity/2)),
    DOMAIN_DISC_CHOICE, DUAL_DISC_CHOICE
)
large_cube = discretize(
    shapes.cube(length=length_wall, h=element_size,
                origin=(-length_wall/2, -length_wall/2, -length_wall/2)),
    DOMAIN_DISC_CHOICE, DUAL_DISC_CHOICE
)

In [4]:
x = large_cube[DOMAIN_DISC_CHOICE]
print(
    x.domain_dimension,
    x.global_dof_count,
    x.flat_local_dof_count,
    sep='\n'
)
y = small_cube[DOMAIN_DISC_CHOICE]
print(
    y.domain_dimension,
    y.global_dof_count,
    y.flat_local_dof_count,
    sep='\n'
)
print()
print(x.global_dof_count+y.global_dof_count)
print(x.flat_local_dof_count+y.flat_local_dof_count)

2
390
780
2
126
252

516
1032


In [5]:
wave_length_w = 2 * np.pi/k['w']
wave_length_w

6.283185307179586

In [6]:
wave_length_e = 2 * np.pi/k['e']
wave_length_e

6.283185307179586

In [7]:
wave_length_w/element_size

12.566370614359172

# Setup the Operator

$$
\mathcal{A}
=
\begin{bmatrix}
    -(\mathcal{A}^w_1 + \mathcal{A}^1_1) & \mathcal{A}^w_{1,w} \\
    -\mathcal{A}^w_{w,1}               & \mathcal{A}^w_w + \mathcal{A}^e_w \\
\end{bmatrix}
$$

In [8]:
def to_block_op(mfie, efie, k, mu):
    """
    Build the standard block operator from the given integral equations.
    """
    A = assembly.BlockedOperator(2, 2) # empty operator object
    A[0,0] = mfie
    A[0,1] = mu/k * efie
    A[1,0] = -k/mu * efie
    A[1,1] = mfie
    return A


def get_simple_block_op(space, k, mu):
    """
    Return a 2x2 block operator defining the block matrix that would
    act on the given grid.
    
    This is similar to the `multitrace_operator` constructor, but it
    allows us to specify exactly which boundary disretisation functions
    to use.
    """
    efie = maxwell.electric_field(
        space[DOMAIN_DISC_CHOICE], space[DUAL_DISC_CHOICE], space[DUAL_DISC_CHOICE], k,
    )
    mfie = maxwell.magnetic_field(
        space[DOMAIN_DISC_CHOICE], space[DUAL_DISC_CHOICE], space[DUAL_DISC_CHOICE], k,
    )
    A = to_block_op(mfie, efie, k, mu)
    return A


def get_mixed_block_op(target, source, k, mu):
    """
    Return a 2x2 block operator that defines the interferences on
    `grid_a` by `grid_b`.
    """
    efie = maxwell.electric_field(
        source[DOMAIN_DISC_CHOICE], target[DUAL_DISC_CHOICE], target[DUAL_DISC_CHOICE], k
    )
    mfie = maxwell.magnetic_field(
        source[DOMAIN_DISC_CHOICE], target[DUAL_DISC_CHOICE], target[DUAL_DISC_CHOICE], k
    )
    A = to_block_op(mfie, efie, k, mu)
    return A
    

def assign_inplace_subblock(A, a, i, j):
    """
    Assigns the 4 elements of a to the 2x2 block of A
    specified by the indexes i and j.
    """
    bi = 2*i
    bj = 2*j
    A[bi, bj]     = a[0, 0]
    A[bi, bj+1]   = a[0, 1]
    A[bi+1, bj]   = a[1, 0]
    A[bi+1, bj+1] = a[1, 1]
    

def PMCHWT_operator(
        small_cube, large_cube, k_int, k_wal, k_ext, mu, parameters
    ):
    """Vector = namedtuple('Vector', ['neumann', 'dirichlet'])
    Build the operator for the nested shape problem formulation.
    """
    # small cube
    A1_1 = get_simple_block_op(small_cube, k_int, mu)
    Aw_1 = get_simple_block_op(small_cube, k_wal, mu)
    # large cube
    Aw_w = get_simple_block_op(large_cube, k_wal, mu)
    Ae_w = get_simple_block_op(large_cube, k_ext, mu)
    # mixed (target, source, k)
    Aw_1w = get_mixed_block_op(small_cube, large_cube, k_wal, mu)
    Aw_w1 = get_mixed_block_op(large_cube, small_cube, k_wal, mu)
   
    # assemble the blocks
    A = assembly.BlockedOperator(2 * 2, 2 * 2)
    assign_inplace_subblock(A, -(Aw_1 + A1_1), 0, 0)
    assign_inplace_subblock(A,   Aw_1w,        0, 1)
    assign_inplace_subblock(A, - Aw_w1,        1, 0)
    assign_inplace_subblock(A,   Aw_w + Ae_w,  1, 1)
    
    return A

    
A = PMCHWT_operator(small_cube, large_cube, k['i'], k['w'], k['e'], mu, None)

In [9]:
spaces_in_A = [d.flat_local_dof_count for d in A._range_spaces]
print(*[d for d in A._domain_spaces], sep='\n')
print(spaces_in_A)
print(sum(spaces_in_A))

[2904, 2904, 9312, 9312]
24432


# Discretize the incident field

In [10]:
#
# Antigoni's code
#
if False:
    def incident_field(x):
        return np.array([0. * x[2], 0. * x[2], np.exp(1j * k_ext * x[0])])


    def dirichlet_trace_fun(x, n, domain_index, result):
        result[:] = np.cross(incident_field(x), n)


    def curl(incident_field,x):
        return np.array([0,  - 1j * k_ext * np.exp(1j * k_ext * x[0]), 0])


    def neumann_trace_fun(x, n, domain_index, result):
        result[:] = (1/(1j * k_ext)) * np.cross(curl(incident_field,x), n)


    incident_field_RWG = number_of_scatterers * [None]
    for i in range(number_of_scatterers):
        dirichlet_trace_RWG = bempp.api.GridFunction(rwg_space[i], fun=dirichlet_trace_fun)
        neumann_trace_RWG = (k_ext/mu_ext) * bempp.api.GridFunction(rwg_space[i], fun = neumann_trace_fun)
        incident_field_RWG[i] = dirichlet_trace_RWG.coefficients.tolist() +  neumann_trace_RWG.coefficients.tolist()

In [11]:
def incident_field(x):
    return np.array([0. * x[2], 0. * x[2], np.exp(1j * k['e'] * x[0])])


def dirichlet_trace_fun(x, n, domain_index, result):
    result[:] = np.cross(incident_field(x), n)

    
def curl(incident_field,x):
    return np.array([0,  - 1j * k['e'] * np.exp(1j * k['e'] * x[0]), 0])


def neumann_trace_fun(x, n, domain_index, result):
    result[:] = (1/(1j * k['e'])) * np.cross(curl(incident_field,x), n)


# for large cube
Dtrace_inc = bempp.api.GridFunction(large_cube[DOMAIN_DISC_CHOICE], fun=dirichlet_trace_fun)
Ntrace_inc = (k['e']/mu) * bempp.api.GridFunction(large_cube[DOMAIN_DISC_CHOICE], fun=neumann_trace_fun)
# combine
Vector = namedtuple('Vector', ['neumann', 'dirichlet'])
u_inc = Vector(
    Dtrace_inc.coefficients.tolist(),
    Ntrace_inc.coefficients.tolist()
)

In [12]:
# theta = np.pi / 4 # Incident wave travelling at a 60 degree angle
# direction = np.array([np.cos(theta), 0, np.sin(theta)])
# polarization = np.array([0, 1.0, 0])

# def plane_wave(point):
#     return polarization * np.exp(1j * k_ext * np.dot(point, direction))

# def dirichlet_trace_fun(point, n, domain_index, result):
#     result[:] =  np.cross(plane_wave(point), n)

# def plane_wave_curl(point):
#     return np.cross(direction, polarization) * 1j * k_ext * np.exp(1j * k_ext * np.dot(point, direction))

# def neumann_trace_fun(point, n, domain_index, result):
#     result[:] =  1./ (1j * k_ext) * np.cross(plane_wave_curl(point), n)

In [13]:
print(len(u_inc[0]))
print(len(u_inc[1]))

390
390


# Assemble the RHS

$$
f = \begin{bmatrix}
    - \mathcal{A_{1w}} \\
    - (\mathcal{A_{w}} - \frac{1}{2} \mathcal{I})
\end{bmatrix}
$$

In [14]:
def get_identity_op(space):
    """
    Create an identity operator matching the given space
    """
    i = bempp.api.operators.boundary.sparse.identity(
        # this can make the kernel crash of not set correctly
        space[DOMAIN_DISC_CHOICE], space[DOMAIN_DISC_CHOICE], space[DOMAIN_DISC_CHOICE]
    )
    I = assembly.BlockedOperator(2, 2)
    I[0, 0] = i
    I[1, 1] = i
    return I
    

Aw_1w = get_mixed_block_op(small_cube, large_cube, k['w'], mu)
Aw_w  = get_simple_block_op(large_cube, k['w'], mu)
I = get_identity_op(large_cube)

In [15]:
print([d.flat_local_dof_count for d in Aw_1w._domain_spaces])
print([d.flat_local_dof_count for d in Aw_1w._range_spaces])
print([d.flat_local_dof_count for d in Aw_w._domain_spaces])
print([d.flat_local_dof_count for d in Aw_w._range_spaces])
print(I._domain_spaces[0].flat_local_dof_count)

[780, 780]
[2904, 2904]
[780, 780]
[9312, 9312]
780


In [16]:
pre = [
    - Aw_1w.weak_form() * (u_inc[0] + u_inc[1]),
    - (Aw_w.weak_form() - 1/2 * I.weak_form()) * (u_inc[0] + u_inc[1])
]
# flatten list of lists
f = [y for x in pre for y in x]

# Solve the Linear System

In [17]:
from antigoni.login import gmres

In [ ]:
# In the last run this stoped convergin around i=4000
# consider setting maxiter to this number for testing
x, info, residuals = gmres(A.weak_form(), f, return_residuals=True, maxiter=2000)

# Plot

## Residual

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.semilogy(residuals);

## Split result into each domain and then into Neumann and Dirichlet traces

In [ ]:

# Split between different domains in vector
u_cavity = Vector(
    neumann=x[:small_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count//2],
    dirichlet=x[small_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count//2:small_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count]
)

split_point = small_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count

u_scattered = Vector(
    neumann=x[split_point:split_point+large_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count//2],
    dirichlet=x[split_point+large_cube[DOMAIN_DISC_CHOICE].flat_local_dof_count//2:]
)

# Check
def assert_lengths_match(a, b):
    """
    Raise Error if lengths of a and b do not match.
    """
    if isinstance(a, int):
        va = a
    else:
        va = len(a)
    
    if isinstance(b, int):
        vb = b
    else:
        vb = len(b)
        
    assert va == vb, "Lengths must match. Got %i and %i." % (va, vb)

    
assert_lengths_match(
    sum([len(z) for z in [
        u_cavity.neumann, u_cavity.dirichlet,
        u_scattered.neumann, u_scattered.dirichlet
    ]]),
    f
)

assert_lengths_match(*u_cavity)
assert_lengths_match(*u_scattered)

## Get the trace of the field in the wall from the scattered and incident traces

In [ ]:
u_wall = Vector(
    u_inc.neumann + u_scattered.neumann,
    u_inc.dirichlet + u_scattered.dirichlet# divide accordingly to achieve smaller particlesrichlet,
)

## Define the points where we will evaluate the field

In [ ]:
# Number of points in the x-direction
nx = 220# divide accordingly to achieve smaller particles

# Number of points in the y-direction
nz = 220

xmin, xmax, zmin, zmax = [
    -length_wall/2-1,
    length_wall/2+1,
    -length_wall/2-1,
    length_wall/2+1
] 
# Ask Antigoni, why j's
plot_grid = np.mgrid[xmin:xmax:nx * 1j, 0:0:1j, zmin:zmax:nz * 1j]

c = 0 # Intersecting plane

points = np.vstack((
    plot_grid[0].ravel(),
    c * np.ones(plot_grid[0].size),
    plot_grid[2].ravel()
))

# Compute interior and exterior indices
all_indices = np.ones(points.shape[1], dtype='uint32')

In [ ]:
def point_is_within_cube(upper_bound_length, point):
    """
    Determines whether the given point is contained within the a centered cube with the
    bounds specified.
    """
    c =  [-upper_bound_length/2 < px and px < upper_bound_length/2
          for px in point]
    return all(c)
        
cavity_indexer = []
wall_indexer = []
exterior_indexer = []

for i, point in enumerate(points.T):
    if point_is_within_cube(length_cavity, point):
        cavity_indexer.append(i)
    elif point_is_within_cube(length_wall, point):
        wall_indexer.append(i)
    elif point_is_within_cube(np.inf, point):
        exterior_indexer.append(i)
    else:
        raise ValueError("Point %s not within domain" % point)

In [ ]:
# Sanity check
print(
    len(cavity_indexer), '+',
    len(wall_indexer), '+',
    len(exterior_indexer), '=',
    sum([
        len(cavity_indexer),
        len(wall_indexer),
        len(exterior_indexer)
    ])
)

## Calculate the field form the trace data

In [ ]:
# Cavity
cavity_points = points[:, cavity_indexer]

E_potential_op = maxwell_potential.electric_field(
    small_cube[DOMAIN_DISC_CHOICE], cavity_points, k['i'])
H_potential_op = maxwell_potential.magnetic_field(
    small_cube[DOMAIN_DISC_CHOICE], cavity_points, k['i'])

Ntrace_i = bempp.api.GridFunction(
    small_cube[DOMAIN_DISC_CHOICE], coefficients=u_cavity.neumann)
Dtrace_i = bempp.api.GridFunction(
    small_cube[DOMAIN_DISC_CHOICE], coefficients=u_cavity.dirichlet)

cavity_field = H_potential_op * Dtrace_i + E_potential_op * (mu/k['i'] * Ntrace_i)

In [ ]:
# Wall
wall_points = points[:, wall_indexer]

# Influence of external boundary
E_potential_op_w = maxwell_potential.electric_field(
    large_cube[DOMAIN_DISC_CHOICE], wall_points, k['w'])
H_potential_op_w = maxwell_potential.magnetic_field(
    large_cube[DOMAIN_DISC_CHOICE], wall_points, k['w'])

Ntrace_w = bempp.api.GridFunction(
    large_cube[DOMAIN_DISC_CHOICE], coefficients=u_wall.neumann)
Dtrace_w = bempp.api.GridFunction(
    large_cube[DOMAIN_DISC_CHOICE], coefficients=u_wall.dirichlet)

# Influence of cavity
E_potential_op_i = maxwell_potential.electric_field(
    small_cube[DOMAIN_DISC_CHOICE], wall_points, k['w'])
H_potential_op_i = maxwell_potential.magnetic_field(
    small_cube[DOMAIN_DISC_CHOICE], wall_points, k['w'])

# Putting them together
wall_field = H_potential_op_w * Dtrace_w + E_potential_op_w * (mu/k['w'] * Ntrace_w) \
    - (H_potential_op_i * Dtrace_i + H_potential_op_i * (mu/k['i'] * Ntrace_i))

In [ ]:
# Scattered
exterior_points = points[:, exterior_indexer]

E_potential_op = maxwell_potential.electric_field(
    large_cube[DOMAIN_DISC_CHOICE], exterior_points, k['e'])
H_potential_op = maxwell_potential.magnetic_field(
    large_cube[DOMAIN_DISC_CHOICE], exterior_points, k['e'])

# TO DO include scaling here
Ntrace = bempp.api.GridFunction(
    large_cube[DOMAIN_DISC_CHOICE], coefficients=u_scattered.neumann)
Dtrace = bempp.api.GridFunction(
    large_cube[DOMAIN_DISC_CHOICE], coefficients=u_scattered.dirichlet)

scattered_field =  - H_potential_op * Dtrace - E_potential_op * (mu/k['e'] * Ntrace)

# Combine, Shape and Plot the Results

In [ ]:
total_field = np.empty_like(points, dtype='complex128')

In [ ]:
total_field[:, cavity_indexer] = cavity_field
total_field[:, wall_indexer] =   wall_field
total_field[:, exterior_indexer] = scattered_field + incident_field(points[:, exterior_indexer])

In [ ]:
# squared_scattered_field = np.sum(np.abs(scattered_field)**2, axis=0)
squared_field = np.sum(np.abs(total_field)**2, axis=0)

In [ ]:
plt.imshow(
    squared_field.reshape(nx, nz),
    extent=[xmin, xmax, zmin, zmax],
    clim = (0,2),
)
plt.colorbar();